In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
data = pd.read_csv("/kaggle/input/heart-failure-prediction/heart.csv")

In [3]:
data.head()

In [4]:
data.shape

In [5]:
data = data.drop_duplicates(keep='first')
data.shape

### Label encoding for changing string value to Numeric

In [6]:
from sklearn.preprocessing import LabelEncoder
data.head(2)

In [7]:
li = ['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope']
for i in li:
    data[i] = LabelEncoder().fit_transform(data[i])

### Find Correlation among Data 

In [8]:
sns.heatmap(data.corr(),annot=True,fmt ='.2f',mask = np.triu(data.corr()))

In [9]:
import warnings
warnings.filterwarnings('ignore')

# visualizationS

In [10]:
plt.figure(figsize=(15,15))
for i in enumerate(data.columns):
    plt.subplot(3,4,i[0]+1)
    sns.scatterplot(data[i[1]],data['MaxHR'])
    

# Analysis For searching any kind of clue

In [39]:
for i in data.columns[:-1]:
    print(i+'\n')
    print(data.groupby('HeartDisease')[i].describe())

# looking for outliers

In [12]:
plt.figure(figsize=(15,15))
for i in enumerate(data.columns):
    plt.subplot(3,4,i[0]+1)
    sns.boxplot(data[i[1]])
    

# Remove Outliers

In [13]:
def remove(data):
  Q1 = np.percentile(data, 25, interpolation = 'midpoint')
  
# Third quartile (Q3)
  Q3 = np.percentile(data, 75, interpolation = 'midpoint')
  iqr = Q3 - Q1
  percentile25 = data.quantile(0.25)
  percentile75 = data.quantile(0.75)

  upper_limit = percentile75 + 1.5 * iqr
  lower_limit = percentile25 - 1.5 * iqr
  data = np.where(
    data > upper_limit,upper_limit,
    np.where(
        data < lower_limit,
        lower_limit,
        data
    )
  )
  return data

#feature[feature['discount_percent'] > upper_limit]
#feature[feature['discount_percent'] < lower_limit]



In [14]:
data.columns

In [15]:
li =['RestingBP','Cholesterol','MaxHR','Oldpeak']
for i in li:
    data[i]=remove(data[i])

# fresh data

In [16]:
plt.figure(figsize=(15,15))
for i in enumerate(data.columns):
    plt.subplot(3,4,i[0]+1)
    sns.boxplot(data[i[1]])
    

# searching Do data need any kind of transformation

In [17]:
plt.figure(figsize=(15,15))
for i in enumerate(data.columns):
    plt.subplot(3,4,i[0]+1)
    sns.distplot(data[i[1]])
    

In [18]:
feature = data.drop('HeartDisease',axis=1)
label = data['HeartDisease']

# Splitting Data

In [19]:
from sklearn.model_selection import train_test_split
trainF,testF,trainL,testL = train_test_split(feature,label,random_state=42,test_size=.2)

# choosing model ,train, test, evalute

In [20]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(trainF,trainL)
pred = model.predict(testF)

In [21]:
from sklearn.metrics import classification_report
print(classification_report(pred,testL))

In [22]:
label.value_counts()

In [23]:
from sklearn.preprocessing import StandardScaler 

In [24]:
li = ['Age','RestingBP','Cholesterol','MaxHR','Oldpeak']
for i in li:
    feature[i] = StandardScaler().fit_transform(feature[[i]])

In [25]:
plt.figure(figsize=(15,15))
for i in enumerate(data.columns):
    plt.subplot(3,4,i[0]+1)
    sns.distplot(data[i[1]])
    

# transform *** "Oldpeak" *** Column 

In [53]:
for i in [2,3,4,5]:
    x = data['Oldpeak']**(1/i)
    sns.distplot(x)
    plt.show()
    
data['Oldpeak'] = data['Oldpeak']**(1/3)

In [70]:
data['HeartDisease'].value_counts()

# We have seen Data is kind of imbalanced.
# real data provides maximum number of patients had Heatdisease

In [59]:
from imblearn.combine  import SMOTEENN
os = SMOTEENN(random_state = 42)
feature,label = os.fit_resample(feature,label)

In [60]:
from sklearn.model_selection import train_test_split
trainF,testF,trainL,testL = train_test_split(feature,label,random_state=42,test_size=.2)

In [61]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(trainF,trainL)
pred = model.predict(testF)
from sklearn.metrics import classification_report
print(classification_report(pred,testL))

# Isn't that geat improvement after balance 1's and 0's data?

# CrossValidation

In [62]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[x**2 for x in range(3,10)],
    'n_jobs':[-1,None],
    "random_state":[42,None]    
}
cv = GridSearchCV(model,params,cv=5)
cv.fit(trainF,trainL)

In [63]:
print(cv.best_score_)

> # tring new Boosting Algorithm Which is **CATBOOSTCLASSIFIER**

In [64]:
import catboost as cb

In [30]:
traindata =cb.Pool(trainF,trainL) 
testdata = cb.Pool(testF,testL)

In [65]:
model = cb.CatBoostClassifier(random_state=42,objective= 'CrossEntropy',)

In [66]:
model.fit(trainF,trainL)

In [67]:
pred = model.predict(testF)

In [68]:
from sklearn import metrics

In [69]:
model.score(testF,testL)

In [71]:
print(metrics.classification_report(testL,model.predict(testF)))

# This is much improver version I must say..
# Randomforest gives us 98% accuracy Score for who had heart disease,  where CAT provides 99%
# Random forest gives 96% recall for who had heat disease, on the other hand CAT is showing 100% *